# Preprocess

Download Squad Dataset, preprocess and save as CSV files

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ls '/content/gdrive/My Drive'

In [ ]:
!pip install datasets==1.0.2
!pip install tqdm==4.55.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.3/68.3 kB 397.6 kB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.1
    Uninstalling tqdm-4.66.1:
      Successfully uninstalled tqdm-4.66.1


In [ ]:
!mkdir '/content/gdrive/My Drive/t5'
!mkdir '/content/gdrive/My Drive/t5/dataset'
!mkdir '/content/gdrive/My Drive/t5/model'
!mkdir '/content/gdrive/My Drive/t5/tokenizer'

mkdir: cannot create directory ‘/content/gdrive/My Drive/t5’: File exists
mkdir: cannot create directory ‘/content/gdrive/My Drive/t5/dataset’: File exists
mkdir: cannot create directory ‘/content/gdrive/My Drive/t5/model’: File exists
mkdir: cannot create directory ‘/content/gdrive/My Drive/t5/tokenizer’: File exists


In [ ]:
!ls '/content/gdrive/My Drive/t5'

dataset  model	tokenizer


In [ ]:
from datasets import load_dataset
train_dataset = load_dataset('squad', split='train')
valid_dataset = load_dataset('squad', split='validation')


Downloading:   0%|          | 0.00/1.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/955 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.12M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

|          | 0/0 [00:00<?, ? examples/s]

|          | 0/0 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/1244d044b266a5e4dbd4174d23cb995eead372fbca31a03edc3f8a132787af41. Subsequent calls will reuse this data.


Reusing dataset squad (/root/.cache/huggingface/datasets/squad/plain_text/1.0.0/1244d044b266a5e4dbd4174d23cb995eead372fbca31a03edc3f8a132787af41)


In [ ]:
len(train_dataset)

87599

In [ ]:
len(valid_dataset)

10570

In [ ]:
len(train_dataset) + len(valid_dataset)

98169

In [ ]:
from pprint import pprint
sample_validation_dataset = next(iter(valid_dataset))
pprint(sample_validation_dataset)

{'answers': {'answer_start': [177, 177, 177],
             'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos']},
 'context': 'Super Bowl 50 was an American football game to determine the '
            'champion of the National Football League (NFL) for the 2015 '
            'season. The American Football Conference (AFC) champion Denver '
            'Broncos defeated the National Football Conference (NFC) champion '
            'Carolina Panthers 24–10 to earn their third Super Bowl title. The '
            "game was played on February 7, 2016, at Levi's Stadium in the San "
            'Francisco Bay Area at Santa Clara, California. As this was the '
            '50th Super Bowl, the league emphasized the "golden anniversary" '
            'with various gold-themed initiatives, as well as temporarily '
            'suspending the tradition of naming each Super Bowl game with '
            'Roman numerals (under which the game would have been known as '
            '"Super

In [ ]:
context = sample_validation_dataset['context']
question = sample_validation_dataset['question']
answer = sample_validation_dataset['answers']['text'][0]
print("context: ", context)
print("question: ", question)
print("answer: ", answer)

context:  Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.
question:  Which NFL team represented the AFC at Super Bowl 50?
answer:  Denver Broncos


In [ ]:
import pandas as pd

In [ ]:
pd.set_option('display.max_colwidth', -1)
from tqdm.notebook import tqdm
df_train = pd.DataFrame(columns = ['context', 'answer', 'question'])
df_validation = pd.DataFrame(columns = ['context', 'answer', 'question'])
print(df_train)
print(df_validation)

Empty DataFrame
Columns: [context, answer, question]
Index: []
Empty DataFrame
Columns: [context, answer, question]
Index: []


<ipython-input-10-ef29f245c15e>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [ ]:
count_long = 0
count_short = 0

for index, val in enumerate(tqdm(train_dataset)):
    passage = val['context']
    question = val['question']
    answer = val['answers']['text'][0]
    no_of_words = len(answer.split())
    if no_of_words >= 7:
      count_long = count_long + 1
      continue
    else:
      df_train.loc[count_short] = [passage] + [answer] + [question]
      count_short = count_short + 1

print("count_long validation dataset: ", count_long)
print("count_short validation dataset: ", count_short)



  0%|          | 0/87599 [00:00<?, ?it/s]

count_long validation dataset:  8935
count_short validation dataset:  78664


In [ ]:
count_long = 0
count_short = 0

for index, val in enumerate(tqdm(valid_dataset)):
    passage = val['context']
    question = val['question']
    answer = val['answers']['text'][0]
    no_of_words = len(answer.split())
    if no_of_words >= 7:
      count_long = count_long + 1
      continue
    else:
      df_validation.loc[count_short] = [passage] + [answer] + [question]
      count_short = count_short + 1

print("count_long validation dataset: ", count_long)
print("count_short validation dataset: ", count_short)

  0%|          | 0/10570 [00:00<?, ?it/s]

count_long validation dataset:  918
count_short validation dataset:  9652


In [ ]:
print(df_train)

In [ ]:
from sklearn.utils import shuffle
df_train = shuffle(df_train)
df_validation = shuffle(df_validation)

In [ ]:
print(df_train.shape)
print(df_validation.shape)

(78664, 3)
(9652, 3)


In [ ]:
df_train.head()

,context,answer,question
67355,"Puerto Rico has been discussed as a potential 51st state of the United States. In a 2012 status referendum a majority of voters, 54%, expressed dissatisfaction with the current political relationship. In a separate question, 61% of voters supported statehood (excluding the 26% of voters who left this question blank). On December 11, 2012, Puerto Rico's legislature resolved to request that the President and the U.S. Congress act on the results, end the current form of territorial status and begin the process of admitting Puerto Rico to the Union as a state.",61%,What percentage of voters supported statehood?
39425,"Near the northern Greek borders there are also some Slavic–speaking groups, locally known as Slavomacedonian-speaking, most of whose members identify ethnically as Greeks. Their dialects can be linguistically classified as forms of either Macedonian Slavic or Bulgarian. It is estimated that after the population exchanges of 1923, Macedonia had 200,000 to 400,000 Slavic speakers. The Jewish community in Greece traditionally spoke Ladino (Judeo-Spanish), today maintained only by a few thousand speakers. Other notable minority languages include Armenian, Georgian, and the Greco-Turkic dialect spoken by the Urums, a community of Caucasus Greeks from the Tsalka region of central Georgia and ethnic Greeks from southeastern Ukraine who arrived in mainly Northern Greece as economic migrants in the 1990s.",Greco-Turkic,The Urums are known to speak what dialect?
9100,Ethnically and linguistically the population of the Republic of the Congo is diverse—Ethnologue recognises 62 spoken languages in the country—but can be grouped into three categories. The Kongo are the largest ethnic group and form roughly half of the population. The most significant subgroups of the Kongo are Laari in Brazzaville and Pool regions and Vili around Pointe-Noire and along the Atlantic coast. The second largest group are the Teke who live to the north of Brazzaville with 17% of the population. Boulangui (M’Boshi) live in the northwest and in Brazzaville and form 12% of the population. Pygmies make up 2% of Congo's population.,62,How many languages are used in the Congo?
72734,"From 1974 (the creation of the British colony of Tuvalu) until independence, the legislative body of Tuvalu was called the House of the Assembly or Fale I Fono. Following independence in October 1978 the House of the Assembly was renamed the Parliament of Tuvalu or Palamene o Tuvalu. The unicameral Parliament has 15 members with elections held every four years. The members of parliament select the Prime Minister (who is the head of government) and the Speaker of Parliament. The ministers that form the Cabinet are appointed by the Governor General on the advice of the Prime Minister.",Parliament of Tuvalu,"After independence, what was the Assembly on Tuvalu named?"
64897,"Neptune's more varied weather when compared to Uranus is due in part to its higher internal heating. Although Neptune lies over 50% further from the Sun than Uranus, and receives only 40% its amount of sunlight, the two planets' surface temperatures are roughly equal. The upper regions of Neptune's troposphere reach a low temperature of 51.8 K (−221.3 °C). At a depth where the atmospheric pressure equals 1 bar (100 kPa), the temperature is 72.00 K (−201.15 °C). Deeper inside the layers of gas, the temperature rises steadily. As with Uranus, the source of this heating is unknown, but the discrepancy is larger: Uranus only radiates 1.1 times as much energy as it receives from the Sun; whereas Neptune radiates about 2.61 times as much energy as it receives from the Sun. Neptune is the farthest planet from the Sun, yet its internal energy is sufficient to drive the fastest planetary winds seen in the Solar System. Depending on the thermal properties of its interior, the heat left over from Neptune's formation may be sufficient to explain its current heat flow, though it is more diffic

In [ ]:
df_validation.head()

,context,answer,question
7614,"The fourth Yuan emperor, Buyantu Khan (Ayurbarwada), was a competent emperor. He was the first Yuan emperor to actively support and adopt mainstream Chinese culture after the reign of Kublai, to the discontent of some Mongol elite. He had been mentored by Li Meng, a Confucian academic. He made many reforms, including the liquidation of the Department of State Affairs (Chinese: 尚書省), which resulted in the execution of five of the highest-ranking officials. Starting in 1313 the traditional imperial examinations were reintroduced for prospective officials, testing their knowledge on significant historical works. Also, he codified much of the law, as well as publishing or translating a number of Chinese books and works.",1313,When did Buyantu resume testing potential government employees?
4293,"Ctenophora (/tᵻˈnɒfərə/; singular ctenophore, /ˈtɛnəfɔːr/ or /ˈtiːnəfɔːr/; from the Greek κτείς kteis 'comb' and φέρω pherō 'carry'; commonly known as comb jellies) is a phylum of animals that live in marine waters worldwide. Their most distinctive feature is the ‘combs’ – groups of cilia which they use for swimming – they are the largest animals that swim by means of cilia. Adults of various species range from a few millimeters to 1.5 m (4 ft 11 in) in size. Like cnidarians, their bodies consist of a mass of jelly, with one layer of cells on the outside and another lining the internal cavity. In ctenophores, these layers are two cells deep, while those in cnidarians are only one cell deep. Some authors combined ctenophores and cnidarians in one phylum, Coelenterata, as both groups rely on water flow through the body cavity for both digestion and respiration. Increasing awareness of the differences persuaded more recent authors to classify them as separate phyla.",marine waters,Where do ctenophora live?
5200,"The glass collection covers 4000 years of glass making, and has over 6000 items from Africa, Britain, Europe, America and Asia. The earliest glassware on display comes from Ancient Egypt and continues through the Ancient Roman, Medieval, Renaissance covering areas such as Venetian glass and Bohemian glass and more recent periods, including Art Nouveau glass by Louis Comfort Tiffany and Émile Gallé, the Art Deco style is represented by several examples by René Lalique. There are many examples of crystal chandeliers both English, displayed in the British galleries and foreign for example Venetian (attributed to Giuseppe Briati) dated c1750 are in the collection. The stained glass collection is possibly the finest in the world, covering the medieval to modern periods, and covering Europe as well as Britain. Several examples of English 16th-century heraldic glass is displayed in the British Galleries. Many well-known designers of stained glass are represented in the collection including, from the 19th century: Dante Gabriel Rossetti, Edward Burne-Jones and William Morris. There is also an example of Frank Lloyd Wright's work in the collection. 20th-century designers include Harry Clarke, John Piper, Patrick Reyntiens, Veronica Whall and Brian Clarke.",René Lalique,The Art Deco style of glassware is represented by which artist?
4283,"In 2010 the Amazon rainforest experienced another severe drought, in some ways more extreme than the 2005 drought. The affected region was approximate 1,160,000 square miles (3,000,000 km2) of rainforest, compared to 734,000 square miles (1,900,000 km2) in 2005. The 2010 drought had three epicenters where vegetation died off, whereas in 2005 the drought was focused on the southwestern part. The findings were published in the journal Science. In a typical year the Amazon absorbs 1.5 gigatons of carbon dioxide; during 2005 instead 5 gigatons were released and in 2010 8 gigatons were released.",1.5 gigatons,How many tons of carbon are absorbed the Amazon in a typical year?
5495,"In the early 1970s, ABC completed its transition to color; the decade as a whole would mark a turning poin

In [ ]:
train_save_path = '/content/gdrive/My Drive/t5/dataset/squad_t5_train_csv'
validation_save_path = '/content/gdrive/My Drive/t5/dataset/squad_t5_val_csv'
df_train.to_csv(train_save_path, index= False)
df_validation.to_csv(validation_save_path, index= False)

# **Testing**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!sudo apt-get update
!sudo apt-get install -y build-essential libffi-dev

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
libffi-dev is already the newest version (3.4.2-4).
0 upgraded, 0 newly insta

In [ ]:
!pip install --quiet tokenizers

In [ ]:
!pip install --quiet transformers
!pip install --quiet pytorch-lightning
!pip install --quiet tokenizers
!pip install --quiet sentencepiece
!pip install --quiet tqdm

In [ ]:
# !pip install torchtext==0.8.0 torch==1.7.1 pytorch-lightning==1.2.2

In [ ]:
import os

def restart_runtime():
  os.kill(os.getpid(), 9)

restart_runtime()

In [ ]:
train_file_path = '/content/gdrive/My Drive/t5/dataset/squad_t5_train_csv'
validation_file_path = '/content/gdrive/My Drive/t5/dataset/squad_t5_val_csv'

In [ ]:
!pip install torch pytorch-lightning

In [ ]:
# !pip install git+https://github.com/PyTorchLightning/pytorch-lightning
# import pytorch_lightning as pl
# print(pl.__version__)

In [ ]:
# !pip install git+https://github.com/PyTorchLightning/pytorch-lightning fsspec --no-deps --target=$nb_path

In [ ]:
# !pip install torchtext==0.8.0 torch==1.7.1 pytorch-lightning==1.2.2


In [ ]:
!pip install https://github.com/PyTorchLightning/pytorch-lightning/releases/download/1.2.6/pytorch-lightning-1.2.6.tar.gz


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install torchtext==0.6.0

In [ ]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
import pandas as pd
import numpy as np
from itertools import chain
from string import punctuation
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from termcolor import colored
import textwrap
from transformers import (
    AdamW,
    AdamWeightDecay,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)
pl.seed_everything(42)

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


42

In [ ]:
t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')
t5_model = T5ForConditionalGeneration.from_pretrained('t5-base')

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
from pprint import pprint
sample_encoding = t5_tokenizer.encode_plus("My name is Pratham Yashwante",
                                           max_length=64,
                                           pad_to_max_length=True,
                                           truncation=True,
                                           return_tensors="pt")
print(sample_encoding.keys())
print(sample_encoding)

dict_keys(['input_ids', 'attention_mask'])
{'input_ids': tensor([[ 499,  564,   19, 6110,   17, 1483,    3,  476, 3198,  210, 1841,    1,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
print(sample_encoding['input_ids'].shape)
print(sample_encoding['input_ids'].squeeze().shape)
print(sample_encoding['input_ids'])

torch.Size([1, 64])
torch.Size([64])
tensor([[ 499,  564,   19, 6110,   17, 1483,    3,  476, 3198,  210, 1841,    1,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0]])


Sentencepeice tokenizer using T5

In [ ]:
tokenized_output = t5_tokenizer.convert_ids_to_tokens(sample_encoding['input_ids'].squeeze())
print(tokenized_output)

['▁My', '▁name', '▁is', '▁Pra', 't', 'ham', '▁', 'Y', 'ash', 'w', 'ante', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


In [ ]:
decoded_output = t5_tokenizer.decode(sample_encoding['input_ids'].squeeze(), skip_special_tokens=True,clean_up_tokenization_spaces=True)
print(decoded_output)

My name is Pratham Yashwante


In [ ]:
print(t5_tokenizer.get_vocab())
print(len(t5_tokenizer.get_vocab().keys()))

{'<pad>': 0, '</s>': 1, '<unk>': 2, '▁': 3, 'X': 4, '.': 5, ',': 6, 's': 7, '▁the': 8, 'a': 9, ':': 10, '▁and': 11, '▁to': 12, '▁of': 13, '▁fill': 14, 'e': 15, '▁in': 16, 't': 17, '-': 18, '▁is': 19, '▁de': 20, '▁for': 21, '’': 22, 'i': 23, '▁that': 24, '▁you': 25, 'd': 26, '▁I': 27, '▁with': 28, 'n': 29, '▁on': 30, "'": 31, 'o': 32, '▁are': 33, '▁it': 34, 'en': 35, '▁be': 36, '▁The': 37, '▁as': 38, '▁your': 39, 'l': 40, '▁(': 41, '▁or': 42, '▁have': 43, '▁at': 44, '▁from': 45, '▁an': 46, '▁was': 47, '▁this': 48, 'er': 49, '▁la': 50, 'm': 51, 'r': 52, 'ing': 53, '▁can': 54, '!': 55, '▁will': 56, '▁by': 57, '?': 58, '▁not': 59, 're': 60, ')': 61, '▁we': 62, 'y': 63, '▁und': 64, '▁has': 65, '▁all': 66, '▁die': 67, '▁but': 68, '▁our': 69, '▁their': 70, '▁A': 71, '▁more': 72, '▁un': 73, '▁der': 74, 'c': 75, 'u': 76, 'in': 77, '▁so': 78, '▁they': 79, '▁one': 80, '▁about': 81, '▁my': 82, 'ul': 83, '▁which': 84, 'à': 85, '▁In': 86, '/': 87, 'he': 88, 'f': 89, '▁le': 90, '▁out': 91, '▁also': 9

Preparing Dataset

In [ ]:
from tqdm.notebook import tqdm
import copy

class QuestionGenerationDataset(Dataset):
  def __init__(self, tokenizer, filepath, max_len_inp=512, max_len_out=96):
    self.path = filepath
    self.passage_column = "context"
    self.answer = "answer"
    self.question = "question"

    self.data = pd.read_csv(self.path, nrows = 1000)

    self.max_len_input = max_len_inp
    self.max_len_output = max_len_out
    self.tokenizer = tokenizer
    self.inputs = []
    self.targets = []
    self.skippedcount = 0
    self._build()

  def __len__(self):

    return len(self.inputs)

  def __getitem__(self, index):
    source_ids = self.inputs[index]["input_ids"].squeeze()
    target_ids = self.targets[index]["input_ids"].squeeze()

    src_mask = self.inputs[index]["attention_mask"].squeeze()
    target_mask = self.targets[index]["attention_mask"].squeeze()

    labels = copy.deepcopy(target_ids)
    labels [labels==0] = -100

    return {"source_ids": source_ids,
            "target_ids": target_ids,
            "source_mask": src_mask,
            "target_mask": target_mask,
            "labels": labels}

  def _build(self):
    for idx in tqdm(range(len(self.data))):
      passage, answer, target = self.data.loc[idx, self.passage_column], self.data.loc[idx, self.answer], self.data.loc[idx, self.question]

      input_ = "context: %s  answer: %s </s>" % (passage, answer)
      target = "question: %s </s>" % (str(target))

      test_input_encoding = self.tokenizer.encode_plus(input_,
                                                        truncation=False,
                                                        return_tensors="pt")

      length_of_input_encoding = len(test_input_encoding['input_ids'][0])

      if length_of_input_encoding > self.max_len_input:
        self.skippedcount = self.skippedcount + 1
        continue

      tokenized_inputs = self.tokenizer.batch_encode_plus(
          [input_], max_length=self.max_len_input, pad_to_max_length=True, return_tensors="pt"
      )

      tokenized_targets = self.tokenizer.batch_encode_plus(
          [target], max_length=self.max_len_output, pad_to_max_length=True, return_tensors="pt"
      )

      self.inputs.append(tokenized_inputs)
      self.targets.append(tokenized_targets)



In [ ]:
train_dataset = QuestionGenerationDataset(t5_tokenizer, train_file_path)

  0%|          | 0/1000 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:303: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g.

In [ ]:
train_sample = train_dataset[50]
decoded_train_input = t5_tokenizer.decode(train_sample['source_ids'])
decoded_train_output = t5_tokenizer.decode(train_sample['target_ids'])

print(decoded_train_input)
print(decoded_train_output)

context: The French planned to invade the British Isles during 1759 by accumulating troops near the mouth of the Loire and concentrating their Brest and Toulon fleets. However, two sea defeats prevented this. In August, the Mediterranean fleet under Jean-François de La Clue-Sabran was scattered by a larger British fleet under Edward Boscawen at the Battle of Lagos. In the Battle of Quiberon Bay on 20 November, the British admiral Edward Hawke with 23 ships of the line caught the French Brest fleet with 21 ships of the line under Marshal de Conflans and sank, captured, or forced many of them aground, putting an end to the French plans. answer: near the mouth of the Loire</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

In [ ]:
validation_dataset = QuestionGenerationDataset(t5_tokenizer, validation_file_path)

  0%|          | 0/1000 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:303: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


T5 Finetuner

In [ ]:
class T5FineTuner(pl.LightningModule):

  def __init__(self, hparams, t5model, t5tokenizer):
    super(T5FineTuner, self).__init__()
    self.hparams = hparams
    self.model = t5model
    self.tokenizer = t5tokenizer

  def forward(self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, lm_labels=None):
    outputs = self.model(
        input_ids = input_ids,
        attention_mask =attention_mask,
        decoder_attention_mask = decoder_attention_mask,
        labels = lm_labels
        )

    return outputs

  def training_step(self, batch, batch_idx):
    outputs = self.forward(
       input_ids = batch["source_ids"],
       attention_mask = batch["source_mask"],
       decoder_input_ids = batch["target_ids"],
       decoder_attention_mask = batch["target_mask"],
       lm_labels = batch['labels']
    )

    loss = outputs[0]
    self.log('train_loss', loss)
    return loss

  def validation_step(self, batch, batch_idx):
    outputs = self.forward(
       input_ids = batch["source_ids"],
       attention_mask = batch["source_mask"],
       decoder_input_ids = batch["target_ids"],
       decoder_attention_mask = batch["target_mask"],
       lm_labels = batch['labels']
    )

    loss = outputs[0]
    self.log('val_loss', loss)

    return loss

  def train_dataloader(self):
    return DataLoader(train_dataset, batch_size=self.hparams.batch_size, num_workers=4)

  def val_dataloader(self):
    return DataLoader(validation_dataset, batch_size=self.hparams.batch_size, num_workers=4)


  # def configure_optimizers(self):
  #   optimizer = AdamW(self.parameters(), lr=3e-4, eps=1e-8)
  #   return optimizer

  def configure_optimizers(self):
      optimizer = torch.optim.Adam(self.parameters(), lr=3e-4, eps=1e-8)
      return [optimizer]

In [ ]:
args_dict = dict(
    batch_size=4,
)

args = argparse.Namespace(**args_dict)

model = T5FineTuner(args, t5_model, t5_tokenizer)

trainer = pl.Trainer(max_epochs= 20, gpus=1, progress_bar_refresh_rate=30)

trainer.fit(model)

print("Saving Model")

save_path_model = '/content/gdrive/My Drive/t5/model/'
save_path_tokenizer = '/content/gdrive/My Drive/t5/tokenizer/'
model.model.save_pretrained(save_path_model)
t5_tokenizer.save_pretrained(save_path_tokenizer)

INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.core.lightning:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Saving Model


('/content/gdrive/My Drive/t5/tokenizer/tokenizer_config.json',
 '/content/gdrive/My Drive/t5/tokenizer/special_tokens_map.json',
 '/content/gdrive/My Drive/t5/tokenizer/spiece.model',
 '/content/gdrive/My Drive/t5/tokenizer/added_tokens.json')

In [ ]:
save_path_model = '/content/gdrive/My Drive/t5/model_new/'
save_path_tokenizer = '/content/gdrive/My Drive/t5/tokenizer_new/'
model.model.save_pretrained(save_path_model)
t5_tokenizer.save_pretrained(save_path_tokenizer)

('/content/gdrive/My Drive/t5/tokenizer_new/tokenizer_config.json',
 '/content/gdrive/My Drive/t5/tokenizer_new/special_tokens_map.json',
 '/content/gdrive/My Drive/t5/tokenizer_new/spiece.model',
 '/content/gdrive/My Drive/t5/tokenizer_new/added_tokens.json')

In [ ]:
!pip install pycm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.8/599.8 kB 9.4 MB/s eta 0:00:00


# Train

In [ ]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install sentencepiece

In [ ]:
trained_model_path = '/content/gdrive/My Drive/t5_new/model/model_new/'
trained_tokenizer = '/content/gdrive/My Drive/t5_new/tokenizer_new/'

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(trained_model_path)
tokenizer = T5Tokenizer.from_pretrained(trained_tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device", device)
model = model.to(device)

device cuda


In [ ]:
context = "Handled MySQL databases to analyze the feedback data and processed it for gaining insights."
answer = "MySQL databases"
text = "context: " + context + " " + "answer: " + answer + " </s>"
print(text)

context: Handled MySQL databases to analyze the feedback data and processed it for gaining insights. answer: MySQL databases </s>


In [ ]:
encoding = tokenizer.encode_plus(text, max_length=512, padding=True, return_tensors="pt")
print(encoding.keys())
input_ids, attention_mask = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)


dict_keys(['input_ids', 'attention_mask'])


In [ ]:
model.eval()
beam_outputs = model.generate(
    input_ids = input_ids ,
    attention_mask = attention_mask,
    max_length=72,
    early_stopping = True,
    num_beams =5,
    num_return_sequences=4
)

for beam_output in beam_outputs:
  sent = tokenizer.decode(beam_output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
  print(sent)

question: What database to analyze feedback data?
question: What databases were used to analyze feedback data?
question: What database used to analyze feedback data?
question: What database helped to analyze feedback data?


In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from tqdm.notebook import tqdm

def calculate_bleu_score(model, tokenizer, context, answer, target):
    text = f"context: {context} answer: {answer} </s>"
    encoding = tokenizer.encode_plus(text, max_length=512, padding=True, return_tensors="pt")
    input_ids, attention_mask = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

    model.eval()
    beam_outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=72,
        early_stopping=True,
        num_beams=5,
        num_return_sequences=4
    )

    generated_questions = [tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True) for output in beam_outputs]

    reference = [target.split()]
    bleu_scores = [sentence_bleu(reference, generated.split()) for generated in generated_questions]

    return max(bleu_scores), generated_questions[bleu_scores.index(max(bleu_scores))]

# Evaluate BLEU score on the validation dataset
total_bleu_score = 0.0
num_samples = 9000
for index in tqdm(range(num_samples)):
    context = validation_dataset.data.loc[index, "context"]
    answer = validation_dataset.data.loc[index, "answer"]
    target = validation_dataset.data.loc[index, "question"]

    bleu_score, generated_question = calculate_bleu_score(model, t5_tokenizer, context, answer, target)
    print(f"Context: {context}\nAnswer: {answer}\nTarget Question: {target}\nGenerated Question: {generated_question}\nBLEU Score: {bleu_score}\n")
    total_bleu_score += bleu_score

average_bleu_score = total_bleu_score / num_samples
print(f"Average BLEU Score: {average_bleu_score}")


  0%|          | 0/9000 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2632: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Context: The fourth Yuan emperor, Buyantu Khan (Ayurbarwada), was a competent emperor. He was the first Yuan emperor to actively support and adopt mainstream Chinese culture after the reign of Kublai, to the discontent of some Mongol elite. He had been mentored by Li Meng, a Confucian academic. He made many reforms, including the liquidation of the Department of State Affairs (Chinese: 尚書省), which resulted in the execution of five of the highest-ranking officials. Starting in 1313 the traditional imperial examinations were reintroduced for prospective officials, testing their knowledge on significant historical works. Also, he codified much of the law, as well as publishing or translating a number of Chinese books and works.
Answer: 1313
Target Question: When did Buyantu resume testing potential government employees?
Generated Question: question: What year did Buyantu Khan begin his reign?
BLEU Score: 6.08970970641905e-155

Context: Ctenophora (/tᵻˈnɒfərə/; singular ctenophore, /ˈtɛnəf

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Context: Another important role of the immune system is to identify and eliminate tumors. This is called immune surveillance. The transformed cells of tumors express antigens that are not found on normal cells. To the immune system, these antigens appear foreign, and their presence causes immune cells to attack the transformed tumor cells. The antigens expressed by tumors have several sources; some are derived from oncogenic viruses like human papillomavirus, which causes cervical cancer, while others are the organism's own proteins that occur at low levels in normal cells but reach high levels in tumor cells. One example is an enzyme called tyrosinase that, when expressed at high levels, transforms certain skin cells (e.g. melanocytes) into tumors called melanomas. A third possible source of tumor antigens are proteins normally important for regulating cell growth and survival, that commonly mutate into cancer inducing molecules called oncogenes.
Answer: melanocytes
Target Question: W

KeyboardInterrupt: ignored

In [ ]:
average_bleu_score = total_bleu_score / num_samples
print(f"Average BLEU Score: {average_bleu_score}")

In [ ]:
############ OG BLEU SCORE ###############

average_bleu_score = total_bleu_score / num_samples
print(f"Average BLEU Score: {average_bleu_score}")

Average BLEU Score: 0.10322998399048526


In [ ]:
!pip install nltk

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge
from tqdm.notebook import tqdm

# Function ROUGE score
def calculate_rouge_score(target, generated):
    rouge = Rouge()

    # Convert target and generated to lowercase
    target_lower = target.lower()
    generated_lower = generated.lower()

    rouge_scores = rouge.get_scores(generated_lower, target_lower, avg=True)
    return rouge_scores

total_rouge_scores = {"rouge-1": 0.0, "rouge-2": 0.0, "rouge-l": 0.0}

num_samples = 3000

for index in tqdm(range(num_samples)):
    context = validation_dataset.data.loc[index, "context"]
    answer = validation_dataset.data.loc[index, "answer"]
    target = validation_dataset.data.loc[index, "question"]

    _, generated_question = calculate_bleu_score(model, t5_tokenizer, context, answer, target)

    rouge_scores = calculate_rouge_score(target, generated_question)

    print(f"Context: {context}\nAnswer: {answer}\nTarget Question: {target}\nGenerated Question: {generated_question}")
    print("ROUGE Scores:", rouge_scores)
    print()

    total_rouge_scores["rouge-1"] += rouge_scores["rouge-1"]["f"]  # Accumulate F1 scores
    total_rouge_scores["rouge-2"] += rouge_scores["rouge-2"]["f"]
    total_rouge_scores["rouge-l"] += rouge_scores["rouge-l"]["f"]

average_rouge_scores = {key: value / num_samples for key, value in total_rouge_scores.items()}

print(f"Average ROUGE Scores:")
print(average_rouge_scores)


  0%|          | 0/1000 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
ROUGE Scores: {'rouge-1': {'r': 0.6363636363636364, 'p': 0.875, 'f': 0.7368421003878117}, 'rouge-2': {'r': 0.5454545454545454, 'p': 0.8571428571428571, 'f': 0.6666666619135801}, 'rouge-l': {'r': 0.6363636363636364, 'p': 0.875, 'f': 0.7368421003878117}}

Context: "Southern California" is not a formal geographic designation, and definitions of what constitutes southern California vary. Geographically, California's north-south midway point lies at exactly 37° 9' 58.23" latitude, around 11 miles (18 km) south of San Jose; however, this does not coincide with popular use of the term. When the state is divided into two areas (northern and southern California), the term "southern California" usually refers to the ten southern-most counties of the state. This definition coincides neatly with the county lines at 35° 47′ 28″ north latitude, which form the northern borders of San Luis Obispo, Kern, and San Bernardino counties. Another definition 

In [ ]:
print(f"Average ROUGE Scores:")
print(average_rouge_scores)

Average ROUGE Scores:
{'rouge-1': 0.4352921346298253, 'rouge-2': 0.23559368506066353, 'rouge-l': 0.41115477837273684}


Average BLEU Score: 0.10322998399048526

Average ROUGE Scores:
{'rouge-1': 0.4352921346298253, 'rouge-2': 0.23559368506066353, 'rouge-l': 0.41115477837273684}

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from nltk.translate.meteor_score import meteor_score
from tqdm.notebook import tqdm

def calculate_meteor_score(model, tokenizer, context, answer, target):
    text = f"context: {context} answer: {answer} </s>"
    encoding = tokenizer.encode_plus(text, max_length=512, padding=True, return_tensors="pt")
    input_ids, attention_mask = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

    model.eval()
    beam_outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=72,
        early_stopping=True,
        num_beams=5,
        num_return_sequences=4
    )

    generated_questions = [tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True) for output in beam_outputs]
    reference = target
    meteor_scores = [meteor_score([reference.split()], generated.split()) for generated in generated_questions]

    max_meteor_score_index = meteor_scores.index(max(meteor_scores))
    generated_question = generated_questions[max_meteor_score_index]

    return max(meteor_scores), generated_question

total_meteor_score = 0.0
num_samples = 3000
for index in tqdm(range(num_samples)):
    context = validation_dataset.data.loc[index, "context"]
    answer = validation_dataset.data.loc[index, "answer"]
    target = validation_dataset.data.loc[index, "question"]

    meteor_score_value, generated_question = calculate_meteor_score(model, t5_tokenizer, context, answer, target)
    print(f"Sample {index + 1} - Context: {context}\nAnswer: {answer}\nTarget Question: {target}\nGenerated Question: {generated_question}\nMETEOR Score: {meteor_score_value}\n")
    total_meteor_score += meteor_score_value

average_meteor_score = total_meteor_score / num_samples
print(f"Average METEOR Score: {average_meteor_score}"

  0%|          | 0/3000 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
Target Question: Where did the movement that would become The United Methodist Church begin?
Generated Question: question: Where did the United Methodist Church begin?
METEOR Score: 0.5964109781843773

Sample 173 - Context: "Southern California" is not a formal geographic designation, and definitions of what constitutes southern California vary. Geographically, California's north-south midway point lies at exactly 37° 9' 58.23" latitude, around 11 miles (18 km) south of San Jose; however, this does not coincide with popular use of the term. When the state is divided into two areas (northern and southern California), the term "southern California" usually refers to the ten southern-most counties of the state. This definition coincides neatly with the county lines at 35° 47′ 28″ north latitude, which form the northern borders of San Luis Obispo, Kern, and San Bernardino counties. Another definition for southern California uses Point Conc

KeyError: ignored

In [ ]:
average_meteor_score = total_meteor_score / num_samples
print(f"Average METEOR Score: {average_meteor_score}")